                                 SAR data processing using snappy module

**Load Python Modulus**

In [1]:
# importing required modules        # Description

import matplotlib.colors as colors  # create visualizations
import matplotlib.image as mpimg    # create visualizations
import matplotlib.pyplot as plt     # create visualizations
from termcolor import colored       # prints colored text
from zipfile import ZipFile         # zip file manipulation
from os.path import join            # data access in file manager
from glob import iglob              # data access in file manager
import pandas as pd                 # data analysis and manipulation
import numpy as np                  # scientific computing
import subprocess                   # external calls to system
import snappy                       # SNAP python interface
# import jpy                        # Python-Java bridge 

**User-defined functions(declaring user defined function to avoid code duplications)**

In [2]:
def output_view(product, band, min_value_VV, max_value_VV, min_value_VH, max_value_VH):
    '''
    Creates visualization of processed Sentinel-1 SAR data
    
    Keywords arguments
    product      -- snappy GPF product --> input Sentinel-1 product
    band         -- List --> product's band to be visualized
    min_value_VV -- int  --> min value for color strech in VV band
    max_value_VV -- int  --> max value for color strech in VV band
    min_value_VH -- int  --> min value for color strech in VH band
    max_value_VH -- int  --> max value for color strech in VH band
    '''
    band_data_list = []
    
    for i in band:
        band = product.getBand(i)
        w = band.getRasterWidth()
        h = band.getRasterHeight()
        band_data = np.zeros(w*h, np.float32)
        band.readPixels(0, 0, w, h, band_data)
        band_data.shape = h, w
        band_data_list.append(band_data)
        
    fig , (ax1, ax2) = plt.subplot(1,2, figsize=(16,16))
    ax1.imshow(band_data_list[0], cmap='grey', vmin=min_value_VV, vmax=max_value_VV)
    ax1.set.title(output_bands[0])
    ax2.imshow(band_data_list[1], cmap='grey', vmin=min_value_VH, vmax=max_value_VH)
    ax2.set.title(output_bands[1])
    
    for ax in fig.get_axes():
        ax.label_outer()
        

**SNAP EO Data Processors are implemented as GPF operators and can be invoked by Windows command-line using the GPF Graph Processing Tool (GPT) which can be found in the bin directory of SNAP installation**

In [3]:
# Call gpt -h from command line
print(subprocess.Popen(['gpt', '-h'], stdout=subprocess.PIPE, universal_newlines=True).communicate()[0])

Usage:
  gpt <op>|<graph-file> [options] [<source-file-1> <source-file-2> ...]

Description:
  This tool is used to execute SNAP raster data operators in batch-mode. The
  operators can be used stand-alone or combined as a directed acyclic graph
  (DAG). Processing graphs are represented using XML. More info about
  processing graphs, the operator API, and the graph XML format can be found
  in the SNAP documentation.

Arguments:
  <op>               Name of an operator. See below for the list of <op>s.
  <graph-file>       Operator graph file (XML format).
  <source-file-i>    The <i>th source product file. The actual number of source
                     file arguments is specified by <op>. May be optional for
                     operators which use the -S option.

Options:
  -h                 Displays command usage. If <op> is given, the specific
                     operator usage is displayed.
  -e                 Displays more detailed error messages. Displays a stack
         

**Read SAR Product**

In [ ]:
#  set the target folder and extract metadata

product_path = "C:\SAR data/"
input_S1_files = sorted(list(iglob(join(product_path, '**', '*S1*.zip'), recursive = True)))

name, sensing_mode, product_type, polarization, height, width, band_names = ([] for i in range(7))

for i in input_S1_files:
    sensing_mode.append(i.split("_")[3])
    product_type.append(i.split("_")[4])
    polarization.append(i.split("_")[-6])
    # Read with Snappy
    s1_read = snappy.ProductIO.readProduct(i)
    name.append(s1_read.getName())
    height.append(s1_read.getSceneRasterHeight())
    width.append(s1_read.getSceneRasterWidht())
    band_names.append(s1_read.getBandNames())
    
df_s1_read = pd.DataFrame({'Name': name, 'Sensing Mode': sensing_mode, 'Product Type': product_type, 'Polarization': polarization, 'Width': width, 'Available Bands': available_bands})
display(df_s1_read)


#Display quicklook image
with ZipFile(input_S1_files[0], 'r') as qck_look:
    qck_look = qck_look.open(name[0] + '.SAFE/preview/quick-look.png')
    img = mpimg.imread(qck_look)
    plt.figure(figure = (15,15))
    plt.title('Quicklook visualization - '+ name[0] + '\n')
    plt.axis('off')
    plt.imshow(img);

**Subset**

In [ ]:
# creating a subset
x, y, width, height = 12000, 8000, 5500, 5500

# Subset Operator -snappy
parameters = snappy.HashMap()
parameters.put('copyMetadata', True)
parameters.put('region', "%s, %s, %s, %s" %(x, y, width, height))
subset = snappy.GPF.createProduct('Subset', parameters, s1_read)
list(subset.getBandNames())

# Plot subset (follow VV -VH order)
output_bands = ['Amplitude_VV', 'Amplitude_VH']
output_view(subset, output_bands, 41, 286, 20, 160)


**Apply Orbit File**

In [ ]:
# Apply Orbit File Operator - snappy
parameters = snappy.HashMap()
parameters.put('Apply-Orbit-file', True)
apply_orbit = snappy.GPF.createProduct('Apply-Orbit-file', parameters, subset)
print(colored('Orbit updated successfully', 'green'))


**Thermal Noise Removal**

In [ ]:
# Thermal Noise Removal Operator -snappy
parameters = snappy.HashMap()
parameters.put('removeThermalNoise', True)
thermal_noise = snappy.GPF.createProduct('ThermalNoiseRemoval', parameters, apply_orbit)

# Plot Thermal Noise Removal (follow VV -VH order)
output_bands = ['Intensity_VV', 'Intensity_VH']
output_view(thermal_noise, output_bands, 0.02, 99376.52, 0.27, 18471.83)

**Radiometric Calibration**

In [ ]:
# Calibration Operator - snappy
parameters = snappy.HashMap()
parameters.put('outputSigmaBand', True)
parameters.put('sourceBands', 'Intensity_VH,Intensity_VV')
parameters.put('selectedPolarisations', 'VH,VV')
Terrain Correction
calibrated = snappy.GPF.createProduct('Calibration', parameters, thermal_noise)

# Plot Calibration (follow VV -VH order)
output_bands = ['Sigma0_VV', 'Sigma0_VH']
output_view(calibrated, output_bands, 0.00, 0.28, 0.00, 0.05)


**Speckle Filtering**

In [ ]:
# Speckle Filtering Operator - snappy
parameters = snappy.HashMap()
parameters.put('filter', 'Lee')
parameters.put('filterSizeX', 5)
parameters.put('filterSizeY', 5)
speckle = snappy.GPF.createProduct('Speckle-Filter', parameters, calibrated)

# Plot speckle filter (follow VV -VH order)
output_bands = ['Sigma0_VV', 'Sigma0_VH']
output_view(speckle, output_bands, 0.00, 0.28, 0.00, 0.05)

**Terrain Correction**

In [ ]:
proj = '''PROJCS["UTM Zone 44/ World Geodetic System 1984",
    GEOGCS["World Geodetic System 1984",
      DATUM["World Geodetic System 1984",
        SPHEROID["WGS 84, 6378137.0, 298.257223563, AUTHORITY["EPSG", "7030"]],
        AUTHORITY["EPSG", "6326"]],
      PREMEM["Greenwich", 0.0, AUTHORITY["EPSG", "8901"]],
      UNIT["degree", 0.017453292519943295],
      AXIS["Geodetic longitude", EAST],
      AXIS["Geodetic latitide", NORTH]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["central_meridian", 3.0],
    PARAMETER["latitude_of_origin", 0.0],
    PARAMETER["scale_factor", 0.9996],
    PARAMETER["false_easting", 500000.0],
    PARAMETER["false_northing", 0.0],
    UNIT["m", 1.0],
    AXIS["Easting", EAST],
    AXIS["Northing", NORTH]]'''
    
# Terrain Correction Operator - snappy
parameters = snappy.HashMap()
parameters.put('demName', 'SRTM 3Sec')
parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
parameters.put('pixelSpacingInMeter', '10')
parameters.put('mapProjection', proj)
parameters.put('nodataValueAtSea', False) # do not mask areas without elevation
parameters.put('saveSelectedSourceBand', True)
terrain_correction = snappy.GPF.createProduct('Terrain-Correction', parameters, speckle)

# Plot terrain correction (follow VV -VH order)
output_bands = ['Sigma0_VV', 'Sigma0_VH']
output_view(terrain_correction, output_bands, 0.00, 0.49, 0.00, 0.04)

**Write the pre-processed SAR product to file in GeoTIFF format**

In [ ]:
# Set output path and name
outpath_name = '/SAR data/Processing/snappy/20210418_Orb_Thm_Cal_Spk_TC'

# Write Operator - snappy
snappy.ProductIO.writeProduct(terrain_correction, output_name, 'GeoTiff')
print(colored('Product successfully saved in:', 'green'), outpath_name)